# Платформа Airbnb в Берлине

Таблица `listings` содержит полную информацию о каждом объявлении Airbnb, включая описания, характеристики и средние оценки по отзывам. Таблица содержит много столбцов, только нужные перечислены в задачах.

Таблица `calendar_summary` предоставляет информацию о доступности и ценах на жильё по дням.

- **listing_id**: Идентификатор объявления
- **date**: Дата
- **available**: Доступность жилья в указанный день (t/f)
- **price**: Цена за ночь

Таблица `reviews` включает отзывы по объявлениям.

- **listing_id**: Идентификатор объявления
- **id**: Идентификатор отзыва
- **date**: Дата отзыва
- **reviewer_id**: Идентификатор автора отзыва
- **reviewer_name**: Имя автора отзыва
- **comments**: Текст отзыва

### 1. Какой месяц и год регистрации новых хостов на Airbnb оказался самым популярным?
- host_id – идентификатор хозяина
- host_since – дата регистрации как хост

```sql
SELECT toStartOfMonth(toDateOrNull(host_since)) AS max_date
FROM listings
GROUP BY max_date
ORDER BY COUNT(DISTINCT host_id) DESC
LIMIT 1
```

### 2. Какая средняя частота ответа среди обычных хостов и суперхостов на Airbnb?
- host_response_rate – частота ответа
- host_is_superhost – является ли суперхозяином


```sql
SELECT host_is_superhost, 
       AVG(host_response_rate) AS AvgResp
FROM
    (SELECT DISTINCT host_id,
                     host_is_superhost,
                     toInt32OrNull(replaceAll(host_response_rate, '%', '')) AS host_response_rate
    FROM listings) as sub
WHERE host_is_superhost != '' 
GROUP BY host_is_superhost 
```

### 3. Какую среднюю цену за ночь устанавливает каждый хост на Airbnb? Идентификаторы сдаваемого жилья объедините в отдельный массив. 
- host_id – идентификатор хозяина
- id – идентификатор жилья
- price – цена за ночь в конкретном месте


```sql
SELECT host_id,
       AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
       groupArray(id) as ids
FROM listings
GROUP BY host_id
ORDER BY avg_price DESC, host_id DESC
```

### 4. Расчитайте разницу между максимальной и минимальной установленной ценой у каждого хозяина. Укажите топ-10 хостов с наибольшей разницей. 

```sql
SELECT host_id,
       MAX(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) 
       - MIN(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS diff_price
FROM listings
GROUP BY host_id
ORDER BY diff_price DESC
LIMIT 10
```

### 5. Выведите средние значения цены за ночь, размера депозита и цены уборки по типу жилья.
- room_type – тип сдаваемого жилья 
- price – цена за ночь
- security_deposit – залог за сохранность имущества
- cleaning_fee – плата за уборку

```sql
SELECT room_type,
       AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
       AVG(toFloat32OrNull(replaceRegexpAll(security_deposit, '[$,]', ''))) AS avg_deposit,
       AVG(toFloat32OrNull(replaceAll(cleaning_fee, '$', ''))) AS avg_cleaning_fee
FROM listings
GROUP BY room_type
ORDER BY avg_deposit DESC
```

### 6. В каком районе города средняя стоимость за ночь на Airbnb является наименьшей?

- price – цена за ночь
- neighbourhood_cleansed – район/округ города

```sql
SELECT neighbourhood_cleansed,
       AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price
FROM listings
GROUP BY neighbourhood_cleansed
ORDER BY avg_price 
LIMIT 1
```

### 7. В каких районах Берлина средняя площадь жилья, сдаваемого целиком, наибольшая? Определите топ-3 района.

- neighbourhood_cleansed – район
- square_feet – площадь
- room_type – тип (нужный – 'Entire home/apt')


```sql
SELECT neighbourhood_cleansed,
       AVG(toFloat32OrNull(square_feet)) AS avg_ssquare_feet
FROM listings
WHERE room_type = 'Entire home/apt'
GROUP BY neighbourhood_cleansed
ORDER BY avg_square DESC
LIMIT 3
```

### 8. Какая из представленных комнат расположена ближе всего к центру города?

- id – идентификатор жилья (объявления)
- room_type – тип жилья ('Private room')
- latitude – широта
- longitude – долгота
- 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

```sql
SELECT id,
       geoDistance(52.5200, 13.4050, toFloat64OrNull(latitude), toFloat64OrNull(longitude)) as distance
FROM listings
WHERE room_type = 'Private room'
ORDER BY distance 
LIMIT 1
```